# Imports

In [43]:
import math
import warnings
import numpy as np
import matplotlib.pyplot as plt
import random
import warnings
from itertools import chain

%matplotlib inline

---

In [51]:
# ------------------------------------------------------------------------------------------------------------------------------
# FOR FUTURE SPEED IMPROVEMENTS WE COULD WORK WITH ARRAY-COORDINATES? ----------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

class _ccircleArray(np.ndarray):
    '''
    Setters and Getters for a list of circles
    Internal use for code readability
    '''
    def __new__(cls, input_array=(np.nan, np.nan, np.nan)):
        obj = np.asarray(input_array).view(cls)
        return obj
    @property
    def x(self):
        return np.array(self[:,0])
    @x.setter
    def x(self, value):
        self[:,0] = value
    @property
    def y(self):
        return np.array(self[:,1])
    @y.setter
    def y(self, value):
        self[:,1] = value
    
    @property
    def xy(self):
        return np.array(self[:,:2])
    @xy.setter
    def xy(self, value):
        self[:,:2] = value
    
    @property
    def r(self):
        return np.array(self[:,2])
    @r.setter
    def r(self, value):
        self[:,2] = value

        
        
class ccircle(np.ndarray):
    '''
    Setters, Getters and Methods for a single circle
    '''
    def __new__(cls, input_array=(np.nan, np.nan, np.nan)):
        obj = np.asarray(input_array).view(cls)
        return obj
    @property
    def x(self):
        return self[0]
    @x.setter
    def x(self, value):
        self[0] = value
    @property
    def y(self):
        return self[1]
    @y.setter
    def y(self, value):
        self[1] = value
    @property
    def xy(self):
        return np.array(self[:2])
    @xy.setter
    def xy(self, value):
        self[:2] = value
    @property
    def r(self):
        return self[2]
    @r.setter
    def r(self, value):
        self[2] = value
    
    def _rewarn(equation, new_warning):
        try:
            with np.errstate(invalid='raise'):
                return equation
        except FloatingPointError:
            with np.errstate(invalid='ignore'):
                warnings.warn(new_warning)
                return equation
    
    def area(self):
        A = math.pi*self.r**2
        return A
    
    def distance(self,list_of_circles):
        '''
        Compute the Euclidean distance between the centerpoint of one circle with a list of (an)other circle(s)  
        '''
        ccArray = _ccircleArray(list_of_circles)
        return np.sqrt((ccArray.x-self.x)**2+(ccArray.y-self.y)**2)

    
    def intersect(self,list_of_circles):
        '''
        Based on Paul Bourke's work
        Compute the (x,y) intersect points between one circle and a list of (an)other circle(s)  
        '''        
        ccArray = _ccircleArray(list_of_circles)
        n_circles = len(ccArray.x)
        d_array = self.distance(ccArray)
        r0 = self.r
        r1_array = ccArray.r

        try:
            with np.errstate(invalid='raise'):
                a = (r0**2-r1_array**2+d_array**2) / (2*d_array)
        except FloatingPointError:
            with np.errstate(invalid='ignore'):
                warnings.warn('FloatingPointError: circles overlap')
                a = (r0**2-r1_array**2+d_array**2) / (2*d_array)
        try:
            with np.errstate(invalid='raise'):
                h = np.sqrt(r0**2-a**2)
        except FloatingPointError:
            with np.errstate(invalid='ignore'):
                warnings.warn('FloatingPointError: no intersections')
                h = np.sqrt(r0**2-a**2)
        P2 = self.xy+a.reshape(n_circles,1)*(ccArray.xy-self.xy)/d_array.reshape(n_circles,1)
        x_y_diff = ccArray.xy-self.xy
        substraend_addend = h.reshape(n_circles,1)*x_y_diff/d_array.reshape(n_circles,1)
        I1 = np.array(P2+substraend_addend[:,::-1]*[1,-1])
        I2 = np.array(P2+substraend_addend[:,::-1]*[-1,1])
        return np.array(list(zip(I1,I2)))
    
    
    


# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

---

## Beginning of Test area

#### Definitions

In [34]:
cc1 = ccircle([6,12,2])
cc2 = ccircle([7,13,2])
cc3 = ccircle([8,13,2])
cc4 = ccircle([6,13,2])

c1 = circle(6,12,2)
c2 = circle(7,13,2)
c3 = circle(8,13,2)
c4 = circle(6,13,2)

#### Accuracy

In [ ]:
cc1.intersect([cc2,cc3])

In [ ]:
c1.multiIntersect([c2,c3])

#### Speed

In [ ]:
cclist = []
for i in range(100000):
    randnum = random.gauss(0,0.1)
    newcircle = ccircle([cc1.x+randnum,cc1.y+randnum,cc1.r])
    cclist.append(newcircle)
    
%timeit cc1.intersect(cclist)

In [ ]:
clist = []
for i in range(100000):
    randnum = random.gauss(0,0.1)
    newcircle = circle(c1.x+randnum,c1.y+randnum,c1.r)
    clist.append(newcircle)
    
%timeit c1.multiIntersect(clist)

## End of Test area

---

In [26]:
class circle(object):
    
    def __init__(self,x_coordinate,y_coordinate,radius):
        self.x = x_coordinate
        self.y = y_coordinate
        self.r = radius
        
    def __str__(self):
        return str((self.x,self.y,self.r))
        
    def area(self):
        A = math.pi*self.r**2
        return A
        
    def distance(self,circle_2):
        return math.sqrt( (circle_2.x-self.x)**2 + (circle_2.y-self.y)**2 )
    
    def multiDistance(self,circles_list):
        map_result = map(self.distance,circles_list)
        return list(map_result)

    def intersect(self,circle_2):
        # Based on Paul Bourke's work
        d = self.distance(circle_2)
        r0 = self.r
        r1 = circle_2.r
        if (d>r0+r1) | (d<abs(r0-r1)) :
            warnings.warn('no intersection > returning None')
            return None
        if (d==0) & (r0==r1):
            raise OverflowError('overlapping circles > infinite number of solutions')
        a = (r0**2-r1**2+d**2) / (2*d)
        h = math.sqrt(r0**2-a**2)
        P0 = np.array([self.x,self.y])
        P1 = np.array([circle_2.x,circle_2.y])
        P2 = P0+a*(P1-P0)/d
        x_y_diff = P1-P0
        substra_add_end = h*(x_y_diff)/d
        I1 = P2+substra_add_end[::-1]*[1,-1]
        I2 = P2+substra_add_end[::-1]*[-1,1]
        return I1,I2

    def intersectCord(self,circle_2):
        d = self.distance(circle_2)
        r0 = self.r
        r1 = circle_2.r
        a = (1/d)*math.sqrt((-d+r1-r0)*(-d-r1+r0)*(-d+r1+r0)*(d+r1+r0))
        return a
    
    def circularSegment(self,cord):
        a = cord #self.intersectCord(circle_2)
        bR = self.r
        sr = (1/2)*math.sqrt(4*bR**2-a**2)
        h = bR-sr
        bA = bR**2 * math.acos((bR-h)/bR) - (bR-h)*math.sqrt(2*bR*h-h**2)
        return bA
        
    def intersectArea(self,circle_2,show_segments=False):
        a = self.intersectCord(circle_2)
        A_self = self.circularSegment(a)
        A_circle_2 = circle_2.circularSegment(a)
        A_total = A_self+A_circle_2
        if show_segments:
            return [A_total,[A_self,A_circle_2]]
        else:
            return A_total
    
    def includePoint(self,point):
        px = point[0]
        py = point[1]
        result = (px-self.x)**2+(py-self.y)**2<=(self.r)**2
        return result
    
    def multiIntersect(self,circles_list,flat=True):
        map_result = map(self.intersect, circles_list)
        if flat:
            return list(chain.from_iterable(map_result))
        else:
            return list(map_result)
    
    def polygonArea(points_list):
        # Area of 3 or more points
        a=1
    
    def multiIntersectArea(self,circles_list):
        # to do: Identify all intersection points 
        # > From those intersection points, identify those that are contained within all circles
        # If None, the circles do not all intersect into one area
        # If != None, calculate the intersecting Area...
        a=1
        
        
    
    

In [ ]:
c1 = circle(6,12,2)
c2 = circle(8,12,2)
c3 = circle(7,10,2)

p1 = [4,12]

print(
    '\n'
    ,c1.includePoint(p1)
    ,'\n'
    ,c3.multiIntersect([c1,c2])
)
    
fig,ax = plt.subplots()
ax.set_xlim((0, 20))
ax.set_ylim((0, 20))
plt.grid(b=True, which='major', color='grey', linestyle='--')
plt.xticks(np.arange(0, 21, 2.0))
plt.yticks(np.arange(0, 21, 2.0))
for cplot in [c1,c2,c3]:
    c = plt.Circle((cplot.x,cplot.y), cplot.r, color='blue', fill=False)
    ax.add_artist(c)
    
plt.scatter(p1[0],p1[1], c='red')